In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np


In [2]:
TRAIN_DIR = '../images/train'
TEST_DIR = '../images/test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                   image     label
0            ../images/train\angry\0.jpg     angry
1            ../images/train\angry\1.jpg     angry
2           ../images/train\angry\10.jpg     angry
3        ../images/train\angry\10002.jpg     angry
4        ../images/train\angry\10016.jpg     angry
...                                  ...       ...
28816  ../images/train\surprise\9969.jpg  surprise
28817  ../images/train\surprise\9985.jpg  surprise
28818  ../images/train\surprise\9990.jpg  surprise
28819  ../images/train\surprise\9992.jpg  surprise
28820  ../images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                                 image     label
0       ../images/test\angry\10052.jpg     angry
1       ../images/test\angry\10065.jpg     angry
2       ../images/test\angry\10079.jpg     angry
3       ../images/test\angry\10095.jpg     angry
4       ../images/test\angry\10121.jpg     angry
...                                ...       ...
7061  ../images/test\surprise\9806.jpg  surprise
7062  ../images/test\surprise\9830.jpg  surprise
7063  ../images/test\surprise\9853.jpg  surprise
7064  ../images/test\surprise\9878.jpg  surprise
7065   ../images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         ../images/test\angry\10052.jpg
1         ../images/test\angry\10065.jpg
2         ../images/test\angry\10079.jpg
3         ../images/test\angry\10095.jpg
4         ../images/test\angry\10121.jpg
                      ...               
7061    ../images/test\surprise\9806.jpg
7062    ../images/test\surprise\9830.jpg
7063    ../images/test\surprise\9853.jpg
7064    ../images/

In [8]:
from tqdm.notebook import tqdm
from PIL import Image

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode= 'grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


In [10]:
train_features = extract_features(train['image']) 
test_features = extract_features(test['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

  0%|          | 0/7066 [00:00<?, ?it/s]

In [11]:
x_train = train_features/255.0
x_test = test_features/255.0

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [14]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [15]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [16]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

c:\Users\iulia\Excalibur\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 125s 546ms/step - accuracy: 0.2346 - loss: 1.8426 - val_accuracy: 0.2583 - val_loss: 1.8103
Epoch 2/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 123s 543ms/step - accuracy: 0.2501 - loss: 1.8140 - val_accuracy: 0.2625 - val_loss: 1.7833
Epoch 3/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 122s 539ms/step - accuracy: 0.2666 - loss: 1.7649 - val_accuracy: 0.3572 - val_loss: 1.6215
Epoch 4/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 122s 538ms/step - accuracy: 0.3488 - loss: 1.6343 - val_accuracy: 0.4352 - val_loss: 1.4439
Epoch 5/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 121s 538ms/step - accuracy: 0.4113 - loss: 1.5080 - val_accuracy: 0.4761 - val_loss: 1.3522
Epoch 6/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 125s 555ms/step - accuracy: 0.4429 - loss: 1.4415 - val_accuracy: 0.4975 - val_loss: 1.3072
Epoch 7/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 122s 539ms/step - accuracy: 0.4604 - loss: 1.3976 - val_accuracy: 0.5113 - val_loss: 1.2802
Epoch 8/150
226/226 ━━━━━━━━━━━━━━━━━━━━ 122s 539ms/step - accuracy: 

In [20]:
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy * 100:.2f}%")


56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 135ms/step - accuracy: 0.5821 - loss: 1.2147
Loss: 1.080765724182129
Accuracy: 62.96%


In [21]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [22]:
from keras.models import model_from_json

In [23]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [24]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [25]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [26]:
print(model.input_shape)

(None, 48, 48, 1)


In [27]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = load_img(image, color_mode='grayscale', target_size=(48, 48))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0
pred = model.predict(img)
pred_label = label[np.argmax(pred)]
print("model prediction is", pred_label)


original image is of sad


FileNotFoundError: [Errno 2] No such file or directory: 'images/train/sad/42.jpg'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
def ef(image_path):
    img = load_img(image_path, target_size=(48, 48), color_mode='grayscale')
    img_array = img_to_array(img)
    img_array = img_array / 255.0 
    img_array = np.expand_dims(img_array, axis=0)  
    return img_array

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/disgust/299.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/surprise/15.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')